In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import Model,layers
from keras.layers import Activation, Dense, Dropout, Conv2D,Conv2DTranspose, Flatten, MaxPooling2D, Input, Add,BatchNormalization
from keras.models import Sequential
from keras.layers.advanced_activations import ReLU,Softmax

Using TensorFlow backend.


In [2]:
Classifier = 'Atraining_normal Atraining_murmur Atraining_artifact Atraining_extrahls'.split()
for i in Classifier:
    pathlib.Path(f'J:/Mtech/signal processing/project/dataset2/{i}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'J:/Mtech/signal processing/project/dataset2/{i}'):
        songname = f'J:/Mtech/signal processing/project/dataset2/{i}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=256, Fs=2, Fc=0, noverlap=128, cmap='CMRmap', sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'J:/Mtech/signal processing/project/dataset2/New folder/{i}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(76369,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(109452,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(20647,)
(110250,)
(106433,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(1

<Figure size 432x288 with 0 Axes>

In [3]:
import split_folders
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio('J:/Mtech/signal processing/project/dataset2/New folder/', output="./data", seed=1337, ratio=(.8, .2))

Copying files: 124 files [00:00, 361.00 files/s]


In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiz
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
training_set = train_datagen.flow_from_directory(
        './data5/train',
        target_size=(64, 64),
        batch_size=49,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        './data5/val',
        target_size=(64, 64),
        batch_size=13,
        class_mode='categorical',
        shuffle = False )

Found 98 images belonging to 4 classes.
Found 26 images belonging to 4 classes.


In [26]:
def build_model():

    X0 = Input(shape=(64, 64, 3))
        
    X1 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same', name='g1')(X0)
    X1 = BatchNormalization()(X1)
    X1 = ReLU()(X1)
    #64,64,64
    
    X2 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same', name='g2')(X1)
    X2 = BatchNormalization()(X2)
    X2 = ReLU()(X2)
    #32,32,64
    
    X3 = Add()([X2, X1])
    X3 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same', name='g3')(X3)
    X3 = BatchNormalization()(X3)
    X3 = ReLU()(X3)
    #16,16,64
    
    X4 = Add()([X3, X2])
    X4 = Conv2D(64, kernel_size = (3,3), strides = (1,1), padding = 'same', name='g4')(X4)
    X4 = BatchNormalization()(X4)
    #X4 = Add()([X4, X0])
    X4 = ReLU()(X4)
    #8,8,64
    
    #X_middle = Conv2D(3, kernel_size = (1,1), strides = (1,1), padding = 'valid', name='gmiddle')(X4)
    #X_middle = BatchNormalization()(X_middle)
    #X_middle = Add()([X_middle, X0])
    #X_middle = ReLU()(X_middle)
    
    #X5 = Add()([X_middle, X3])
    #X5 = Conv2D(128, kernel_size = (3, 3), strides = (2,2), padding = 'same', name='g5')(X5)
    #X5 = ReLU()(X5)
    #X5 = BatchNormalization()(X5)
    
    #X6 = Add()([X5, X2])
    #X6 = Conv2D(64, kernel_size = (3, 3), strides = (2,2), padding = 'same', name='g6')(X6)
    #X6 = ReLU()(X6)
    #X6 = BatchNormalization()(X6)
    
    #X7 = Add()([X6, X1])
    #X7 = Conv2D(64, kernel_size = (3, 3), strides = (2,2), padding = 'same', name='g7')(X7)
    #X7 = ReLU()(X7)
    #X7 = BatchNormalization()(X7)
    
    X9 = Flatten()(X4)
    
    X10 = Dense(128)(X9)
    X10 = ReLU()(X10)
    
    X11 = Dense(128)(X10)
    X11 = ReLU()(X11)
    
    X12 = Dense(128)(X11)
    X12 = ReLU()(X12)
    
    X13 = Dense(128)(X12)
    X13 = ReLU()(X13)
    
    X14 = Dense(4)(X13)
    X14 = Softmax()(X14)
    
    model = Model(inputs = X0, outputs = X14)
        
    return model

model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
g1 (Conv2D)                     (None, 64, 64, 64)   1792        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 64, 64, 64)   256         g1[0][0]                         
__________________________________________________________________________________________________
re_lu_17 (ReLU)                 (None, 64, 64, 64)   0           batch_normalization_9[0][0]      
__________________________________________________________________________________________________
g2 (Conv2D

In [27]:
model.compile(optimizer="adam", loss="logcosh", metrics=['accuracy'])

In [28]:
model.fit_generator(
        training_set,
        steps_per_epoch=10,
        epochs=10)

Epoch 1/10
10/10 [==============================] - 40s 4s/step - loss: 0.1464 - acc: 0.2939
Epoch 2/10
 2/10 [=====>........................] - ETA: 28s - loss: 0.1461 - acc: 0.3265

KeyboardInterrupt: 

In [35]:
import tensorflow as tf
model=tf.keras.models.load_model('dataset2_final_RCNN_model.h5')

In [10]:
model.evaluate_generator(generator=test_set, steps=50)

[0.07225028425455093, 0.65384614]

In [36]:
output = model.predict_generator(test_set,steps = 2)

In [37]:
output=np.argmax(output,axis=1)

In [38]:
output

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2,
       3, 2, 0, 2], dtype=int64)

In [21]:
true_labels=test_set.classes

In [22]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [39]:
confusion_matrix(true_labels,output)

array([[8, 0, 0, 0],
       [0, 0, 2, 2],
       [0, 0, 7, 0],
       [1, 0, 4, 2]], dtype=int64)

In [40]:
accuracy_score(true_labels,output)

0.6538461538461539

In [41]:
print(classification_report(true_labels,output))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.00      0.00      0.00         4
           2       0.54      1.00      0.70         7
           3       0.50      0.29      0.36         7

    accuracy                           0.65        26
   macro avg       0.48      0.57      0.50        26
weighted avg       0.55      0.65      0.58        26

